In [1]:
#import dwavebinarycsp as dbc
import minorminer
from dwave.system.samplers import DWaveSampler 
import dimod
import statistics as stat
import math
my_solver = 'DW_2000Q_2'
my_token = 'PREM-426a488702253a4cd34901800bf165bb55582e3c'

from pseudobinary import PBP

In [2]:
def print_list(lst, header='', cpad=6, rpad=120):
    line = ''
    for i in range(len(lst))[::-1]:
        if lst[i]:
            line += str(lst[i]).rjust(cpad)
        else:
            line += ''.rjust(cpad)
    line = header.ljust(cpad) + line.rjust(rpad)
    print(line)

modulus = 143
#modulus = 899
#modulus = 3599
#modulus =  4757
#modulus = 14351
#modulus = 376289

N = list()
while modulus > 0:
    N.append(PBP(0,-(modulus%2)))
    modulus >>= 1
if len(N)%2: N.append(PBP(0,0))
    
n = len(N)//2 - 2
p = [PBP()] + [PBP(x) for x in range(1,n+1)] + [PBP()]
q = [PBP()] + [PBP(x) for x in range(n+1,2*n+1)] + [PBP()]
print_list(p, 'p')
print_list(q, 'q')

bit_diff = n
carry_start = 2*n+1

next_var = 2*n+1

mtab = list()
for i in range(len(q)):
    row = [None]*i
    mtab.append(row)
    for pi in p: row.append(pi*q[i])

for i in range(1, len(p)+len(q)-1):
    count = 0
    for row in mtab:
        if len(row) > i and row[i]: count += 1
    carries = math.floor(math.log2(count))
    carry_mult = 2
    for j in range(carries):
        row = [None] * i
        row.append(PBP(next_var, -carry_mult))
        row += [None] * j
        row.append(PBP(next_var))
        next_var += 1
        carry_mult *= 2
        mtab.append(row)
        
mtab.append(N)
        
for row in mtab: print_list(row)
    
equations = list()
for j in range(1,len(N)):
    eqn = PBP(0,0)
    for row in mtab:
        if len(row)>j and row[j]: eqn += row[j]
    eqn.trim()
    equations.append(eqn)
    

for eqn in equations: print(eqn)

p                                                                                                          1    x2    x1     1
q                                                                                                          1    x4    x3     1
                                                                                                           1    x2    x1     1
                                                                                                    x3  x2x3  x1x3    x3      
                                                                                              x4  x2x4  x1x4    x4            
                                                                                         1    x2    x1     1                  
                                                                                                                x5  -2x5      
                                                                                                          x6  -

In [6]:
def test_substitution(eqn, var, value):
    f = eqn.substitute(var, value)
    H = f*f
    if H.degree() > 2: H = H.toQuadratic()
    if H.varcount() > 21:
        print("%d is too many variables" % H.varcount())
        return (True, -1, -1)
    bqm = H.toBQM()
    response = dimod.ExactSolver().sample(bqm)
    return (min(response.data_vectors['energy']) == 0, f.varcount(), H.varcount())

solutions = dict()
degenerate = True

max_vars = 0
max_qubits = 0

eqns = list(equations)
while(False):
    made_change = False
    #check for solvable variable
    for eqn in eqns:
        if eqn.degree() == 1 and eqn.varcount() <= 2:
            var = eqn.variables()[0]
            sub = eqn.solvefor(var)
            
            if sub.varcount() == 1:
                same = sub.const() == 0 and sub.coeff(sub.variables()[0]) == 1
                different = sub.const() == 1 and sub.coeff(sub.variables()[0]) == -1
                if not same and not different: continue
            #if sub.const()<0: continue
            #if sub.const() == 1 and sub.varcount() == 1 and sub.coeff(sub.variables()[0]) != -1: continue
                
            if degenerate and var < carry_start and sub.varcount() == 1:
                var2 = sub.variables()[0]
                if var2 < carry_start and abs(var2-var) == bit_diff and sub.const() == 1:
                    degenerate = False
                    sub = PBP(0,1)
                    solutions[var2] = PBP(0,0)
                    reduced = list()
                    for eqn in eqns:
                        eqn = eqn.substitute(var2, PBP(0,0))
                        if not eqn.iszero(): reduced.append(eqn)
                    eqns = reduced
                    print("Broke degeneracy with x%d and x%d" % (var, var2))
                    
            solutions[var] = sub
            
            reduced = list()
            for eqn in eqns:
                eqn = eqn.substitute(var, sub)
                if not eqn.iszero(): reduced.append(eqn)
            
            eqns = reduced
            made_change = True
            print('Solved for and eliminated x%d' % var)
            for eqn in eqns: print(eqn)
            print("")
            break
            
    if made_change: continue
    
    #check for variables that are determined by single equation
    sorted_eqns = sorted(eqns, key=lambda x:(x.varcount(), x.degree()))
    for eqn in sorted_eqns:
        for var in eqn.variables():
            solution = None
            
            solved, vcount, qcount = test_substitution(eqn, var, PBP(0,0))
            max_vars = max(max_vars, vcount)
            max_qubits = max(max_qubits, qcount)
            if not solved: solution = PBP()
            
            solved, vcount, qcount = test_substitution(eqn, var, PBP())
            max_vars = max(max_vars, vcount)
            max_qubits = max(max_qubits, qcount)
            if not solved: solution = PBP(0,0)
                
            if solution is not None:
                solutions[var] = solution
                reduced = list()
                for x in eqns:
                    x = x.substitute(var, solution)
                    if not x.iszero(): reduced.append(x)
                eqns = reduced
                made_change = True
                print("Solved x%d=" % var, solution, "in ", eqn)
                for x in eqns: print(x)
                print("")
                break
                
        if made_change: break
                
    if not made_change: break

        
for var, solution in solutions.items():
    if solution.varcount() == 1:
        unknown = solution.variables()[0]
        if unknown in solutions and solutions[unknown].varcount()==0:
            solutions[var] = solution.substitute(unknown, solutions[unknown])
    
for key in sorted(solutions.keys()):
    print(key, "=", solutions[key])

print("Max variables: ", max_vars)
print("Max qubits: ", max_qubits)

if set(range(1, carry_start)).issubset(set(solutions.keys())):
    p=1+2**(bit_diff+1)
    for x in range(1,bit_diff+1): p+=solutions[x].const()*2**x
    q=1+2**(bit_diff+1)
    for x in range(1,bit_diff+1): q+=solutions[x+bit_diff].const()*2**x
    print("Solved!  %d * %d = %d" % (p,q, p*q))
else:
    H = PBP(0,0)
    for eqn in eqns: H += eqn*eqn
    print("%d variables remaining" % H.varcount())
    H = H.toQuadratic(mode=0)
    print("%d qubits remaining" % H.varcount())
    bqm = H.toBQM()


Max variables:  0
Max qubits:  0
14 variables remaining
32 qubits remaining


In [5]:
sampler = DWaveSampler(solver=my_solver, token=my_token)
_, target_edgelist, target_adjacency = sampler.structure

embedding = minorminer.find_embedding(bqm.quadratic, target_edgelist)
if bqm and not embedding:
    raise ValueError("no embedding found")
    
bqm_embedded = dimod.embed_bqm(bqm, embedding, target_adjacency, 8)

print(len(list(embedding.keys())), "logical qubits")
phys = list()
for _, v in embedding.items():
    phys += v
print(len(phys), "physical qubits")

chain_lens = [len(v) for _, v in embedding.items()]
print('Longest chain: ', max(chain_lens))
print('Shortest chain: ', min(chain_lens))
print('Average chain: ', stat.mean(chain_lens))

response = sampler.sample(bqm_embedded, num_reads = 1000, answer_mode = 'histogram')
response = dimod.unembed_response(response, embedding, source_bqm=bqm)

row_format ="{:>8}" * 2
print(row_format.format('Energy', 'Count'))
for sample in response.data(fields=['sample', 'energy', 'num_occurrences']):
    print(row_format.format(sample[1], sample[2]))
    print(sample[0])

31 logical qubits
121 physical qubits
Longest chain:  12
Shortest chain:  1
Average chain:  3.903225806451613
  Energy   Count
     0.0       1
{1: 1, 2: 0, 3: 0, 4: 1, 5: 0, 6: 0, 7: 0, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 14: 1, 15: 1, 16: 0, 17: 1, 18: 1, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 1, 25: 1, 26: 1, 27: 1, 28: 1, 29: 1, 30: 1, 31: 0, 32: 0}
     0.0       1
{1: 0, 2: 1, 3: 1, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 14: 1, 15: 1, 16: 1, 17: 0, 18: 1, 19: 0, 20: 0, 21: 1, 22: 1, 23: 1, 24: 0, 25: 0, 26: 0, 27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 0}
     1.0       1
{1: 0, 2: 0, 3: 1, 4: 1, 5: 0, 6: 0, 7: 0, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 14: 1, 15: 1, 16: 0, 17: 1, 18: 1, 19: 0, 20: 0, 21: 0, 22: 1, 23: 0, 24: 0, 25: 1, 26: 0, 27: 1, 28: 1, 29: 1, 30: 1, 31: 0, 32: 0}
     1.0       1
{1: 0, 2: 0, 3: 1, 4: 1, 5: 0, 6: 0, 7: 0, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 0, 20: 0, 21: 1, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 1, 28: 

{1: 0, 2: 0, 3: 1, 4: 1, 5: 0, 6: 1, 7: 0, 8: 0, 9: 1, 10: 1, 11: 0, 12: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 0, 21: 1, 22: 0, 23: 1, 24: 1, 25: 1, 26: 1, 27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 0}
    14.0       1
{1: 0, 2: 0, 3: 1, 4: 1, 5: 0, 6: 1, 7: 0, 8: 0, 9: 1, 10: 1, 11: 0, 12: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 0, 19: 1, 20: 0, 21: 1, 22: 0, 23: 1, 24: 1, 25: 1, 26: 1, 27: 1, 28: 1, 29: 0, 30: 0, 31: 0, 32: 0}
    14.0       1
{1: 1, 2: 0, 3: 1, 4: 0, 5: 1, 6: 0, 7: 0, 8: 1, 9: 0, 10: 1, 11: 1, 12: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 0, 20: 1, 21: 1, 22: 1, 23: 0, 24: 1, 25: 1, 26: 0, 27: 1, 28: 1, 29: 0, 30: 1, 31: 0, 32: 0}
    14.0       1
{1: 0, 2: 0, 3: 1, 4: 1, 5: 0, 6: 1, 7: 0, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 0, 19: 1, 20: 0, 21: 1, 22: 0, 23: 0, 24: 0, 25: 1, 26: 1, 27: 1, 28: 0, 29: 1, 30: 1, 31: 1, 32: 0}
    14.0       1
{1: 1, 2: 1, 3: 1, 4: 0, 5: 1, 6: 0, 7: 1, 8: 1, 9: 0, 10: 1, 11: 1, 12: 1, 14: 1, 15: 1, 16: 1,

In [27]:
#tests
print(PBP(), " should be 1")
print(PBP(0,2), " should be 2")
print(PBP(0,-2), " should be -2")
print(PBP(1), " should be x1")
print(PBP(1,2), " should be 2x1")
print(PBP(15), " should be x1x2x3x4")
print(PBP(0,2)*PBP(15), "should be 2x1x2x3x4")
print(PBP(2)*PBP(4), "should be x2x3")
x1 = PBP(1)
x2 = PBP(2)
print(x1, " should be x1")
print(x2, " should be x2")
print(x1+x2, "should be x1+x2")
print((x1+x2)*(x1+x2), "should be x1+x2+2x1x2")

1  should be 1
2  should be 2
-2  should be -2
x1  should be x1
2x1  should be 2x1
x1x2x3x4  should be x1x2x3x4
2x1x2x3x4 should be 2x1x2x3x4
x2x3 should be x2x3
x1  should be x1
x2  should be x2
x1+x2 should be x1+x2
x1+x2+2x1x2 should be x1+x2+2x1x2


True